In [1]:
%load_ext autoreload
%autoreload 2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline,HuggingFaceHub
import transformers as ts

In [ ]:
from CustomLLM import LocalLLM
mp='/home/xyy/hfdownloader/aliendao-main/dataroot/models/Qwen/Qwen-7B'
local_llm=LocalLLM.from_pretrain(repo_path=mp)
ts.AutoModelForCausalLM.from_pretrained()

In [8]:
_PROMPT_TEMPLATE = """
用户会提出一个需要你回答的问题，你应该对问题进行理解回答。

对于用户的问题，你输出的内容应该是一个一行的字符串，下面是一个例子

例子:
用户：蒙古国的首都是
回答：蒙古国的首都是乌兰巴托（Ulaanbaatar）


你的回答格式应该按照下面的内容，请注意“---output”等标记都必须输出，这是我用来提取答案的标记。
不要输出中文的逗号，不要输出引号。除了格式示例的内容，你不应该输出其他文字。

格式示例：
---input
用户的问题
---output
你的答案


现在，我们开始作答
问题: 中国的首都是？
"""

In [10]:
import dashscope
from dashscope import Generation
from http import HTTPStatus
import random
dashscope.api_key='sk-02b8b1a9175a4cae9bf9571d517d8b81'
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': _PROMPT_TEMPLATE}]
response = Generation.call(
    Generation.Models.qwen_max,
    #messages=messages,
    prompt=_PROMPT_TEMPLATE,
    # set the random seed, optional, default to 1234 if not set
    seed=random.randint(1, 10000),
    result_format='message',  # set the result to be "message" format.
)
if response.status_code == HTTPStatus.OK:
    print(response['output'])
else:
    print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
        response.request_id, response.status_code,
        response.code, response.message
    ))


{"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "---input\n中国的首都是？\n---output\n中国的首都是北京（Beijing）。"}}]}
